In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joypy import joyplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import os
import json
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import combinations
from collections import Counter

## Función para limpiar las herramientas

In [5]:
def safe_literal_eval(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):

        return set()

## Función para contar las herramientas

In [6]:
def contar_herramientas(df):
    # Aplanamos la lista de sets en una sola lista de herramientas
    herramientas_lista = [herramienta for sublist in df['herramientas'].tolist() for herramienta in sublist]
    # Contamos la ocurrencia de cada herramienta
    return pd.Series(herramientas_lista).value_counts()

## Función para asignar rangos

In [7]:
def intervalo_experiencia(experiencia):
    if experiencia <= 1:
        return 'Junior'
    elif experiencia <= 4:
        return 'Mid'
    elif experiencia <= 7:
        return 'Senior'
    elif experiencia <= 10:
        return 'Lead'
    return np.nan

## Función normalizar salarios

In [8]:
def normalizar_salarios_columna(df_original, salario, columnas):
    df = df_original.copy()
    
    df.dropna(subset = salario)
    #Normlaizamos la columna salario
    
    df[salario] = (df[salario] - df[salario].min()) / (df[salario].max() - df[salario].min())

    #Multiplicamos los valores booleanos por la normalización del salario df_herramientas_salarios

    for columna in columnas:
        df[columna] = df[columna] * df[salario]
    
    return df

## Funcion para obtener categoria

In [9]:
def obtener_categoria(row):
    for columna in columnas_booleanas:
        if row[columna] == 1:
            return columna
    return None    

## Funcion para cambiar booleanos

In [10]:
def cambiar_bool_por_salario(df_original, salario, columnas):
    df = df_original.copy()
    
    df.dropna(subset = salario)
    #Multiplicamos los valores booleanos por la normalización del salario df_herramientas_salarios

    for columna in columnas:
        df[columna] = df[columna] * df[salario]
    
    return df

## Funcion para combinar herramientas

In [11]:
def combinaciones_herramientas(df, columnas, n):
    combinaciones_posibles = list(combinations(df[columnas], n))
    dic = {}

    # Contar las combinaciones que más se repiten con valor 1
    contador_combinaciones = Counter()

    for combinacion in combinaciones_posibles:
        # Filtrar filas donde todas las columnas en la combinación tienen valor 1
        filas_con_uno = df[df[list(combinacion)].eq(1).all(axis=1)]
        dic[combinacion] = filas_con_uno.index
        
        # Contar la frecuencia de la combinación
        contador_combinaciones[combinacion] += len(filas_con_uno)

    # Obtener las combinaciones más comunes
    combinaciones_mas_comunes = contador_combinaciones.most_common()
    
    df = pd.DataFrame(combinaciones_mas_comunes, columns = ['combinacion', 'total'])
    
    return df, dic

## Funcion para eliminar outliers

In [ ]:
def metodo_tukey(df, columna, alfa):
    q1 = df[columna].quantile(0.25)
    q3 = df[columna].quantile(0.75)
    riq = q3 - q1

    df = df[df[columna].between(q1 - alfa * riq, q3 + alfa * riq) | (df[columna].isna())]
    
    return df